In [ ]:
#Tạo đường dẫn chung để đọc utils
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [ ]:
# PDF parsing
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTTextLine, LTChar
from pypdf import PdfReader

# Text processing
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
import unicodedata
import json
import os

# Embedding & Vector search
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
import torch
import numpy as np

# Utils
from utils.load_chunks_json import load_chunks_from_json
from utils.save_chunks_json import save_chunks_to_json
from utils.bm25 import bm25_tokenize, text_to_sparse_vector_bm25

# ***Tạo hàm xử lý các chunk***

In [27]:
def parse_chapters(text):
    """
    Phân tích 1 dòng chương/tiêu đề, hỗ trợ các định dạng:
    - 'Chương 1 Tiêu đề...'
    - 'Chương nhập môn Tiêu đề...'
    - 'I. Tiêu đề...'
    - '1. Tiêu đề...'

    Trả về tuple: (chapter_number, chapter_title)
    Nếu không khớp format, trả về (, )
    """
    pattern = re.compile(
        r"""^\s*
        (                                         # Nhóm 1: định danh chương
            Chương\s+(?:nhập\s+môn|\d+)           # 'Chương nhập môn' hoặc 'Chương <số>'
            |[IVXLCDM]+\.                         # Số La Mã có dấu chấm (I., II., ...)
            |\d+\.?                                # Số thường, có thể có hoặc không dấu chấm
        )
        \s*(.*)                                    # Nhóm 2: tiêu đề còn lại
        """, re.IGNORECASE | re.VERBOSE
    )
    
    line = text.strip()
    match = pattern.match(line)
    if match:
        chapter_number = match.group(1).strip().rstrip('.') or ""
        chapter_title = match.group(2).strip() or ""

        if (chapter_number and len(chapter_number) > 1) or (chapter_title and len(chapter_title) > 1):
            return chapter_number, chapter_title
        else:
            return "", text
    else:
        return "", text


In [28]:
def normalize_text(text):
    """
    Chuyển chuỗi thành lowercase, xóa dấu, và xóa toàn bộ khoảng trắng.
    """
    # Lowercase
    text = text.lower()

    # Bỏ dấu
    text = unicodedata.normalize('NFD', text)
    text = ''.join(c for c in text if unicodedata.category(c) != 'Mn')

    # Xóa toàn bộ khoảng trắng (space, tab, newline)
    text = re.sub(r'\s+', '', text)

    return text

In [29]:
# Hàm dùng để chuyển đổi data từ RAW chunks sang Dense Chunks
def raw_to_dense_chunk(
    chapter_line,
    section_line,
    sub_section_line,
    content_line,
    max_chars=2048,
    model= None
):

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=max_chars,
        chunk_overlap=300,  
        separators=["\n\n", "\n", ".", " ", ""]
    )
    content_chunks = splitter.split_text(content_line)

    # Xử lý metadata như cũ
    chapter_number, chapter_title = parse_chapters(chapter_line) if chapter_line else ("", "")
    section_number, section_title = parse_chapters(section_line) if section_line else ("", "")
    subsection_number, subsection_title = parse_chapters(sub_section_line) if sub_section_line else ("", "")

    if 'MỤC TIÊU' in section_line:
        chunk_type = 'TARGET'
    elif 'NỘI DUNG ÔN TẬP VÀ THẢO LUẬN' in section_line:
        chunk_type = 'EXERCISES'
    else:
        chunk_type = 'THEORY'

    result = []
    for i, content_piece in enumerate(content_chunks):
        content = content_piece.strip()
        chunk = {
            "id": f"LSD_{normalize_text(chapter_number) if chapter_number else 0}_{section_number if section_number else 0}_{subsection_number if subsection_number else 0}_{i}",
            "values": model.encode(content).tolist(),
            "metadata": {
                "subject": "Lịch sử Đảng Cộng Sản Việt Nam",
                "chapter": chapter_number,
                "chapter_title": chapter_title,
                "section": section_number,
                "section_title": section_title,
                "subsection": subsection_number,
                "subsection_title": subsection_title,
                "content": content,
                "tokens": len(content_piece),
                "type": chunk_type
            }
        }
        result.append(chunk)

    return result

In [31]:
# Hàm dùng để chuyển đổi data từ RAW chunks sang Sparse Chunks
def raw_to_sparse_chunk(
    chapter_line,
    section_line,
    sub_section_line,
    content_line,
    bm25=None,
    vocabulary=None,
    max_chars=2048
):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=max_chars,
        chunk_overlap=300,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    content_chunks = splitter.split_text(content_line)

    # Metadata parsing
    chapter_number, chapter_title = parse_chapters(chapter_line) if chapter_line else ("", "")
    section_number, section_title = parse_chapters(section_line) if section_line else ("", "")
    subsection_number, subsection_title = parse_chapters(sub_section_line) if sub_section_line else ("", "")

    if 'MỤC TIÊU' in section_line:
        chunk_type = 'TARGET'
    elif 'NỘI DUNG ÔN TẬP VÀ THẢO LUẬN' in section_line:
        chunk_type = 'EXERCISES'
    else:
        chunk_type = 'THEORY'

    result = []
    for i, content_piece in enumerate(content_chunks):
        content = content_piece.strip()
        sparse_vector = text_to_sparse_vector_bm25(content, bm25, vocabulary)

        chunk = {
            "id": f"LSD_{normalize_text(chapter_number) if chapter_number else 0}_{section_number if section_number else 0}_{subsection_number if subsection_number else 0}_{i}",
            "sparse_values": sparse_vector,
            "metadata": {
                "subject": "Lịch sử Đảng Cộng Sản Việt Nam",
                "chapter": chapter_number,
                "chapter_title": chapter_title,
                "section": section_number,
                "section_title": section_title,
                "subsection": subsection_number,
                "subsection_title": subsection_title,
                "content": content,
                "tokens": len(content_piece),
                "type": chunk_type
            }
        }
        result.append(chunk)

    return result


In [32]:
def extract_raw_chunks_LSD(pdf_path, total_pages):
    """
    Trích xuất các đoạn nội dung theo chương - mục - mục con mà chưa split theo embedding.
    Trả về danh sách các dict: {"chapter": ..., "section": ..., "sub_section": ..., "content": ...}
    """
    raw_chunks = []
    is_content = False

    curr_chapter_line = ""
    curr_section_line = ""
    curr_sub_section_line = ""
    curr_content_line = ""
    temp = ["1", "2", "3", "4"]
    count = 0 
    for page_layout in extract_pages(pdf_path, page_numbers=range(2, total_pages)):
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                for text_line in element:
                    if isinstance(text_line, LTTextLine):
                        line_text = ""
                        for obj in text_line:
                            if isinstance(obj, LTChar):
                                char = obj.get_text()
                                size = obj.size

                                if size < 10 and char.isdigit():
                                    continue

                                line_text += char
                        line_text = line_text.strip()
                        # line_text = text_line.get_text().strip()
                        first_obj = next(iter(text_line), None)

                        if isinstance(first_obj, LTChar):
                            font = first_obj.fontname
                            text_size = first_obj.size

                            if text_size == 14:
                                if is_content:
                                    raw_chunks.append({
                                        "chapter": curr_chapter_line.strip(),
                                        "section": curr_section_line.strip(),
                                        "sub_section": curr_sub_section_line.strip(),
                                        "content": curr_content_line.strip()
                                    })
                                    curr_chapter_line = ""
                                    curr_section_line = ""
                                    curr_sub_section_line = ""
                                    curr_content_line = ""
                                    is_content = False
                                curr_chapter_line += line_text + " "

                            elif 12.9 < text_size < 14:
                                if "Bold" in font and "Italic" not in font:
                                    if is_content and line_text not in ["", " ", "c", ","]:
                                        for temp_item in temp:
                                            if temp_item in curr_section_line:
                                                curr_content_line = curr_section_line + curr_content_line
                                                curr_section_line = temp_item
                                                curr_sub_section_line = ""
                                        if "KẾT LUẬN" in curr_section_line.upper():
                                            curr_chapter_line = "KẾT LUẬN"
                                            curr_section_line = ""
                                            curr_sub_section_line = ""
                                        raw_chunks.append({
                                            "chapter": curr_chapter_line.strip(),
                                            "section": curr_section_line.strip(),
                                            "sub_section": curr_sub_section_line.strip(),
                                            "content": curr_content_line.strip()
                                        })
                                        curr_section_line = ""
                                        curr_sub_section_line = ""
                                        curr_content_line = ""
                                        is_content = False
                                    curr_section_line += line_text + " "

                                elif "Bold" in font and "Italic" in font:
                                    if is_content and line_text.strip() != "":
                                        raw_chunks.append({
                                            "chapter": curr_chapter_line.strip(),
                                            "section": curr_section_line.strip(),
                                            "sub_section": curr_sub_section_line.strip(),
                                            "content": curr_content_line.strip()
                                        })
                                        curr_sub_section_line = ""
                                        curr_content_line = ""
                                        is_content = False
                                    curr_sub_section_line += line_text + " "

                                else:
                                    is_content = True
                                    curr_content_line += line_text + " "
    if is_content and curr_content_line.strip():
        raw_chunks.append({
            "chapter": curr_chapter_line.strip(),
            "section": curr_section_line.strip(),
            "sub_section": curr_sub_section_line.strip(),
            "content": curr_content_line.strip()
        })

    return raw_chunks

In [33]:
def chunk_with_embedding(raw_chunks, embedding_model):
    """
    Nhận vào danh sách raw_chunks, sau đó chuyển danh sách các raw chunks sang Dense chunks
    """
    final_chunks = []
    for chunk in raw_chunks:
        split_chunks = raw_to_dense_chunk(
            chunk['chapter'],
            chunk['section'],
            chunk['sub_section'],
            chunk['content'],
            max_chars=2048,
            model=embedding_model
        )
        final_chunks.extend(split_chunks)

    return final_chunks

In [34]:
def chunk_with_sparse(raw_chunks, bm25, vocabulary):
    """
    Nhận vào danh sách raw_chunks, sau đó chuyển danh sách các raw chunks sang Dense chunks
    """
    final_chunks = []
    for chunk in raw_chunks:
        split_chunks = raw_to_sparse_chunk(
            chunk['chapter'],
            chunk['section'],
            chunk['sub_section'],
            chunk['content'],
            bm25=bm25,
            vocabulary=vocabulary
        )
        final_chunks.extend(split_chunks)

    return final_chunks

# ***Main***

In [36]:
pdf_path = "./lich_su_dang.pdf"

# # Lấy số trang trong file PDF
# reader = PdfReader(pdf_path)
# total_pages = len(reader.pages)
# # print("Số trang trong PDF:", total_pages)

# #Trích xuất rawchunks từ file PDF
# LSD_raw_chunk = extract_raw_chunks_LSD(pdf_path, total_pages)

# #Tạo Densve Vector từ RAW chunks
# embedding_model = SentenceTransformer("AITeamVN/Vietnamese_Embedding")
# embedding_model.max_seq_length = 2048
# LSD_dense_chunks = chunk_with_embedding(LSD_raw_chunk, embedding_model)

# #Tạo Sparse Vector từ RAW chunks
# corpus_texts = [chunk["content"] for chunk in LSD_raw_chunk]
# tokenized_corpus = [bm25_tokenize(text) for text in corpus_texts]
# bm25 = BM25Okapi(tokenized_corpus)
# vocabulary = list(bm25.idf.keys())
# LSD_sparse_chunks = chunk_with_sparse(LSD_raw_chunk, bm25, vocabulary)

## Lưu các Vector vừa tạo ra file Json
# save_chunks_to_json(LSD_raw_chunk, r"./Lich_Su_Dang_raw.json")
# save_chunks_to_json(LSD_dense_chunks, r"./Lich_Su_Dang_Dense.json")
# save_chunks_to_json(LSD_sparse_chunks, r"./Lich_Su_Dang_Sparse.json")

#Đọc các file json ra thành chunks, chuẩn bị upsert lên Database
LSD_raw_chunk = load_chunks_from_json(r"./Lich_Su_Dang_raw.json")
LSD_dense_chunks = load_chunks_from_json(r"./Lich_Su_Dang_Dense.json")
LSD_sparse_chunks = load_chunks_from_json(r"./Lich_Su_Dang_Sparse.json")